In [1]:
import os
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

folder = r"F:\DPL\project4\finale\wav2vec2_VN"

from transformers import AutoModelForCTC, AutoTokenizer, AutoFeatureExtractor, Wav2Vec2Processor, AutoConfig

config = AutoConfig.from_pretrained(folder)

tokenizer = AutoTokenizer.from_pretrained(os.path.join(folder, "tokenizer"))
feature_extractor = AutoFeatureExtractor.from_pretrained(os.path.join(folder, "feature_extractor"))
processor = Wav2Vec2Processor.from_pretrained(os.path.join(folder, "processor"))

model = AutoModelForCTC.from_config(config)
model.load_state_dict(torch.load(os.path.join(folder, "model.pth"), map_location=device))
model.to(device)
model.eval()


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [2]:
from datasets import load_dataset, Audio
data = load_dataset("mozilla-foundation/common_voice_11_0", "vi", split="validation[:100]")

In [3]:
data = data.cast_column("audio", Audio(sampling_rate=16_000))

In [4]:
def VietnameseSpeechRecognition(arrTensor):
  inputs = processor(arrTensor, sampling_rate=16000, return_tensors="pt")
  inputs = inputs.to(device)
  with torch.no_grad():
      logits = model(**inputs).logits
      predicted_ids = torch.argmax(logits, dim=-1)
      transcription = processor.batch_decode(predicted_ids)
      return transcription[0]

In [7]:
import IPython.display as ipd
import numpy as np
import random
audio_sample = data[15]["audio"]["array"]
ipd.Audio(data=audio_sample, autoplay=False, rate=16000)

In [ ]:
print(VietnameseSpeechRecognition(audio_sample))

không có hay là anh là ma


In [ ]:
import os

import numpy as np

import gradio as gr
import scipy.signal as sps

def ReSampleAudio(data,current_rate,new_rate=16000):
  samples = round(len(data) * float(new_rate) / current_rate)
  return sps.resample(data, samples)

def reverse_audio(audio):
    sr, dataArr = audio
    dataArr = ReSampleAudio(dataArr,sr)
    inputs = torch.tensor(dataArr)
    return VietnameseSpeechRecognition(inputs)


demo = gr.Interface(fn=reverse_audio,
                    inputs="microphone",
                    outputs="text",
                    title="Nhận diện tiếng Việt",
                    cache_examples=True,
                    )


demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\dat57\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dat57\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dat57\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\dat57\AppData\Local\Packages\Python